# Iceland KH-9 Panoramic Camera preprocessing

In [ ]:
import hipp
import os

## Settings

In [ ]:
# path settings
PROJECT_DIRECTORY = "data/iceland/kh9_pc_images/1980_08_22"
RAW_IMAGES_DIRECTORY = os.path.join(PROJECT_DIRECTORY, "raw_images")
PREPROCESSED_IMAGES_DIRECTORY = os.path.join(PROJECT_DIRECTORY, "preprocessed_images")
JOINED_IMAGES_DIRECTORY = os.path.join(PROJECT_DIRECTORY, "joined_images")
CROPPING_BORDERS_FILE = os.path.join(PROJECT_DIRECTORY, "cropping_borders.csv")

# others settings
QUICKVIEWS_FACTOR = 0.05
MAX_WORKERS = 8

## Downloading the dataset

| Dataset Name | Date | Images Count | Preprocess Notebook | Raw images acces| Preprocessed images size | Raw images size|
|--|--|--|--|--|--|--|
| Casa Grande KH-9 PC | 1978/03/25 | 6 |here| USGS | 43 Go | 47 Go |


In [ ]:
if not os.path.exists(RAW_IMAGES_DIRECTORY):
    !wget -c -r -np -nd -A "*.tif" -P {RAW_IMAGES_DIRECTORY} https://ftp.lmi.is/.stm/joaquin/history/iceland/kh9_pc_images/
    hipp.tools.generate_quickviews(RAW_IMAGES_DIRECTORY, QUICKVIEWS_FACTOR, max_workers=MAX_WORKERS)
else:
    print(f"{RAW_IMAGES_DIRECTORY} already exists, skipping download.")

## Step 2 : Joining Images

The first step of the preprocessing pipeline is to **join multiple image tiles** into a single, continuous image. This process is necessary because **KH9 PC images are typically split into 10-12 separate parts**, each approximately **1 GB in size**, due to their large original resolution.

However, **joining these image parts is not straightforward**, primarily because there is a **slight overlap between adjacent tiles**. 

To accurately reconstruct the full image, we need to perform the following steps:

1. **Detect keypoints (interest points)** along the **right border** of the first image part.
2. Detect corresponding keypoints along the **left border** of the next image part.
3. **Match these keypoints** using a feature-matching algorithm to find candidate correspondences.
4. Use **RANSAC (Random Sample Consensus)** to filter out mismatched or erroneous correspondences (i.e., outliers).
5. Estimate a **relative geometric transformation** (typically a translation or affine transform) to correctly align the second image with the first.
6. Apply this transformation and **merge the two parts** into a larger composite image.

This process is repeated sequentially for all image parts, progressively building up the full image mosaic.

To perform that 2 functions exists:
- `join_images_asp` (recommanded) : which will use [`image_mosaic`](https://stereopipeline.readthedocs.io/en/latest/tools/image_mosaic.html) program from Ames Stereo Pipeline. This function is much more safer but it requiered to install ASP and the command need to be visible in your path.
- `join_images` (beta) : is a pure Python implementation that replicates the same processing steps, but currently lacks blending in overlapping regions.


In [ ]:
# this version use image_mosaic from asp, So you need to have asp in your path
hipp.kh9pc.join_images_asp(RAW_IMAGES_DIRECTORY, JOINED_IMAGES_DIRECTORY)

## Step 3 : Define manualy all cropping points

The function `hipp.kh9pc.select_all_cropping_points` allow you to manually select corners of the region of interest.
- Split each image with 5 x 20 blocks (parameters `grid_shape`)
- open each corners block in an interactive window where you can Ctrl + Click on the corner
- save all results in the csv file (`csv_file`)

**Note** :
- This function dont overwrite existing data in the csv file (if existing)

In [ ]:
hipp.kh9pc.select_all_cropping_points(JOINED_IMAGES_DIRECTORY, CROPPING_BORDERS_FILE)

## Step 4 : Cropping image to remove borders

With the previously generated cropping points, we can use the `hipp.kh9pc.crop_images` to finish the preprocessing.
For each image in the input directory, this function looks up its corresponding cropping points in the CSV file, rotates the image to align the top edge, crops it accordingly, and saves the result in the output directory.

**Note :**

- Images will have different size according to their coresponding croppings points


In [ ]:
hipp.kh9pc.crop_images(JOINED_IMAGES_DIRECTORY, CROPPING_BORDERS_FILE, PREPROCESSED_IMAGES_DIRECTORY)
hipp.tools.generate_quickviews(PREPROCESSED_IMAGES_DIRECTORY, QUICKVIEWS_FACTOR)